Overall the temporal model does not explain any of the temporal variance in the data. There are two potential explanations for this failure. First, the observed temporal variation in the labels originates from misreporting in the surveys and not from actual variation in the wealth indicators. In general survey data on consumption expenditure is quite noisy as it involves recalling what a housheold consumed in a given time period prior to the survey date. Thus, parts of the temporal variation in the data might actually originate from misreporting rather than from actual variation in the wealth indicator. If the ratio of variation due to noise in survey answers to actual variation in the wealth indicator is too large, any model will have a hard time learning meaningful mappings between the features and the target variables. Unfortunately, there is no direct way to estimate this ratio. However, I study different channels that affect noise in the training data. 

First: the level of analysis is the cluster level and the target variables are the average response of all households within a cluster. In general, a cluster level average will be more noisy, if it relies on fewer households. Thus, I reduce noise in the target variables by iteratively excluding clusters with fewer than $H_c \in {1,...,15}$ households. I train a model for each value of $H_c$. 

Second: time of survey as quality indicator.

Another source of noise that might affect the performance of models is introduced by jittered geoloactions. I recenter geolocations using the four-step procedure described in section .... Arguably, clusters that are recentered in the first round of the procedure are geographically closer to their true location. Thus, I study whether it makes a difference if clusters are recentered in either of the four steps, by excluding clusters that are recentered in any of the subsequent iterations. 



However, here I empirically assess 



a statistical argument can be made to study this: the level of analysis is the cluster level and the more households to 

In [1]:
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import pickle

from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

In [2]:
# set the font size for matplotlib and the font family.
font = {'family' : 'sans-serif',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)

In [3]:
# load the necessary functions from the analysis package

# load the variable names, this allows to access the variables in the feature data in a compact way
from analysis_utils.variable_names import *

# load flagged ids 
from analysis_utils.flagged_uids import *

# load the functions to do spatial k-fold CV
from analysis_utils.spatial_CV import *

# load the helper functions
from analysis_utils.analysis_helpers import *

# load the random forest trainer and cross_validator
import analysis_utils.RandomForest as rf

# load the combien model
from analysis_utils.CombinedModel import CombinedModel

In [4]:
# set the global file paths
root_data_dir = "../../Data"

# the lsms data
lsms_pth = f"{root_data_dir}/lsms/processed/labels_cluster_v1.csv"

# the feature data
feat_data_pth = f"{root_data_dir}/feature_data/tabular_data.csv"

# set the random seed
random_seed = 423
spatial_cv_random_seed = 348

# set the number of folds for k-fold CV
n_folds = 5

In [5]:
# load the feature and the label data
lsms_df = pd.read_csv(lsms_pth)

# remove flagged ids form dataset
lsms_df = lsms_df[~lsms_df.unique_id.isin(flagged_uids)].reset_index(drop = True)

# create average varaibles for the between model
lsms_df['avg_log_mean_pc_cons_usd_2017'] = lsms_df.groupby('cluster_id')['log_mean_pc_cons_usd_2017'].transform('mean')
lsms_df['avg_mean_asset_index_yeh'] = lsms_df.groupby('cluster_id')['mean_asset_index_yeh'].transform('mean')

# load the feature data
feat_df = pd.read_csv(feat_data_pth)

# describe the training data broadly
print(f"Number of observations {len(lsms_df)}")
print(f"Number of clusters {len(np.unique(lsms_df.cluster_id))}")
print(f"Number of x vars {len(feat_df.columns)-2}")

Number of observations 6401
Number of clusters 2128
Number of x vars 113


In [6]:
# merge the label and the feature data to one dataset
lsms_vars = ['unique_id','n_households', 'country', 'lat', 'lon',        
             'log_mean_pc_cons_usd_2017', 'avg_log_mean_pc_cons_usd_2017',
             'mean_asset_index_yeh', 'avg_mean_asset_index_yeh']
df = pd.merge(lsms_df[lsms_vars], feat_df, on = 'unique_id', how = 'left')

In [7]:
df = pd.merge(lsms_df, feat_df, on  = ('cluster_id', 'unique_id'), how = 'left')

In [8]:
# define the x variables and the target variable for the between and within models

# define the between variables
avg_rs_vars = avg_ndvi_vars + avg_ndwi_gao_vars + avg_nl_vars
osm_vars = osm_dist_vars + osm_count_vars + osm_road_vars
between_x_vars = osm_vars + esa_lc_vars + wsf_vars + avg_rs_vars + avg_preciptiation + median_rgb_vars
between_target_var = 'avg_log_mean_pc_cons_usd_2017'

# define the within varaibles
dyn_rs_vars = dyn_ndvi_vars + dyn_ndwi_gao_vars + dyn_nl_vars
within_x_vars = dyn_rs_vars + precipitation + dyn_rgb_vars 
within_target_var = 'log_mean_pc_cons_usd_2017'

# Different number of households


In [9]:
# define the housheold thresholds
n_households = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]
# n_households = [1,2]

# define the between df for this task, inlcuding all necessary variables
between_df = df[['cluster_id', between_target_var, 'n_households'] + between_x_vars].drop_duplicates().reset_index(drop = True)

# define the within df for this task, including all necessary variables
within_df = df[['unique_id', 'cluster_id', within_target_var, 'n_households'] + within_x_vars]

In [10]:
# run the training and valdiation

combined_r2 = []
between_r2 = []
within_r2 = []

for i in n_households:
    print(f"Training on at least {i} Households per cluster")
    
    # subset the datasets to only those observations that meet the condition
    between_df_sub = between_df[between_df['n_households'] >= i].reset_index(drop = True)
    within_df_sub = within_df[within_df['n_households'] >= i].reset_index(drop = True)
    df_sub = df[df['n_households'] >= i].reset_index(drop = True)
    
    # standardise the datasets
    between_df_norm = standardise_df(between_df_sub, exclude_cols = [between_target_var])
    demeaned_df_norm = standardise_df(demean_df(within_df_sub), exclude_cols = [within_target_var])
    
    aux_r2_b = []
    aux_r2_w = []
    aux_r2_c = []
    
    # do 10-times repeated CV: 
    for j in range(10):
        seeed = spatial_cv_random_seed + i + j
        # split the data into training and validation folds
        fold_ids = split_lsms_spatial(df_sub, n_folds = n_folds, random_seed = seeed)

        # run the bewtween training
        print('\n\tBetween model')
        between_cv_trainer = rf.CrossValidator(between_df_norm, 
                                               fold_ids,
                                               between_target_var, 
                                               between_x_vars, 
                                               id_var = 'cluster_id',
                                               random_seed = seeed)
        between_cv_trainer.run_cv_training(min_samples_leaf = 1)
        between_res = between_cv_trainer.compute_overall_performance()

        # run the within training
        print('\n\tWithin Model')
        within_cv_trainer = rf.CrossValidator(demeaned_df_norm, 
                                              fold_ids, 
                                              within_target_var, 
                                              within_x_vars,
                                              id_var = 'unique_id',
                                              random_seed = seeed)
        within_cv_trainer.run_cv_training(min_samples_leaf = 15)
        within_res = within_cv_trainer.compute_overall_performance()

        # run the combined model
        combined_model = CombinedModel(df_sub, between_cv_trainer, within_cv_trainer)
        combined_model.evaluate()
        combined_res = combined_model.compute_overall_performance()
        print(f"\t{combined_res['r2']}")

        # store results
        aux_r2_b.append(combined_res['r2']['between'])
        aux_r2_w.append(combined_res['r2']['within'])
        aux_r2_c.append(combined_res['r2']['overall'])
    
    between_r2.append(aux_r2_b)
    within_r2.append(aux_r2_w)
    combined_r2.append(aux_r2_c)
    
    print('\n\n\n')


Training on at least 1 Households per cluster
Fold 0, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 1, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 2, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 3, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 4, specified test ratio: 0.2 - Actual test ratio 0.19

	Between model
Initialising training


  0%|          | 0/5 [00:00<?, ?it/s]

Finished training after 207 seconds

	Within Model
Initialising training


  0%|          | 0/5 [00:00<?, ?it/s]

Finished training after 351 seconds
	{'overall': 0.36359376348725075, 'between': 0.4376299931936972, 'within': 0.006640541326003113}
Fold 0, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 1, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 2, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 3, specified test ratio: 0.2 - Actual test ratio 0.20
Fold 4, specified test ratio: 0.2 - Actual test ratio 0.18

	Between model
Initialising training


...

Finished training after 32 seconds
	{'overall': 0.11457570395367449, 'between': 0.1060314034438688, 'within': 0.007895226136051758}
Fold 0, specified test ratio: 0.2 - Actual test ratio 0.26
Fold 1, specified test ratio: 0.2 - Actual test ratio 0.24
Fold 2, specified test ratio: 0.2 - Actual test ratio 0.18
Fold 3, specified test ratio: 0.2 - Actual test ratio 0.21
Fold 4, specified test ratio: 0.2 - Actual test ratio 0.11

	Between model
Initialising training


  0%|          | 0/5 [00:00<?, ?it/s]

Finished training after 34 seconds

	Within Model
Initialising training


  0%|          | 0/5 [00:00<?, ?it/s]

Finished training after 32 seconds
	{'overall': 0.08408081566141294, 'between': 0.05605824305273629, 'within': -0.03142244265464722}






In [11]:
combined_res

{'r2': {'overall': 0.08408081566141294,
  'between': 0.05605824305273629,
  'within': -0.03142244265464722},
 'mse': {'overall': 0.08137813006923288,
  'between': 0.062018405759575894,
  'within': 0.020053056834449384}}

In [12]:
results = {
    'between_r2': np.array(between_r2),
    'within_r2': np.array(within_r2),
    'combined_r2': np.array(combined_r2)
}

# take the average over the repeated CV results
agg_results = {k: np.mean(v, axis = 1) for k,v in results.items()}

# save the results
pth = "results/robustness/robust_households.pkl"
with open(pth, 'wb') as f:
    pickle.dump(results, f)